In [ ]:
# Imports
import keras
import numpy as np
import matplotlib.pyplot as plt
import glob

%matploblib inline

In [ ]:
# Load test data
test_images_path = 'data/test_images'
test_label_path = 'data/test_labels'
test_images = np.load(test_images_path)
test_labels = np.load(test_label_path)

In [ ]:
# Load model and weights
base_model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, input_shape=IMAGE_DIM+(3,), classes=NUM_CLASSES)
model = keras.models.Model(inputs=base_model.input, outputs=base_model.get_layer(name='fc1000').output)
model.compile(optimizer = keras.optimizers.Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

all_weights = glob.glob()
model.load_weights()